In [247]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [248]:
import random
from Mylib import *
# %matplotlib inline

In [249]:
GSAPI = GgleAPI('credentials.json', 'sheets')
GSAPI.Create_Service('v4', ['https://www.googleapis.com/auth/spreadsheets'])

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=780794052173-vq88qrrbro0ukaan0jltei7g2vpmrhik.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=jI2ZDP4mY7uncxJ3F9Rm0aHDfGMt5Y&access_type=offline
sheets service created successfully


In [319]:
doC = pd.read_pickle('./data/election_21st.pickle')

rms = ['정책', '개혁', '추진', '강화', '확대', '지원', '별첨', '자료', '방안', '공약'] # cliche
doC = {'{}'.format(p):[[w for w in d if w not in rms] for d in c] for p, c in doC.items()}
doC = {'{}'.format(p):[d for d in c if d != []] for p, c in doC.items()} # remove vacant docs

In [320]:
def nrmlzcorp(corpus, thrsh = 7):
    result = [[]]
    for i, d in enumerate(corpus):
        if len(result[-1]) < thrsh:
            result[-1] += d
        else:
            result.append(d)
    return(result)

nrmcorp = nrmlzcorp(my_list([corp for corp in doC.values()]).sum())
len(nrmcorp)

2031

In [327]:
def Main(use_tqdm):
    project = '21대_총선'
    random_seed = random.randint(1, 10**4)
#     random_seed = 2803
    # tpc_grd = [int(i) for i in np.linspace(start = 40, stop = 20, num = 21)] # Remark: decreasing order!
    tpc_grd = [26, 24, 22, 20]
    save = True

    iterators = [(ng,alp) 
                 for ng in [1, 2, 3][1:2]
                 for alp in ['symmetric', 'asymmetric'][1:2]]
    
    if use_tqdm:
        iterators = tqdm(iterators, 'iterations')
    for i in iterators:
        ngram, alpha = i
    #     Testing?
    #     R = random.Random(2020)
    #     doC = {'{}'.format(p):R.sample(c, 30) for p, c in doC.items()}

    #     1.2 ngrams model
        corp_forLDA = make_ngram(nrmcorp, ngram, mincnt = 1, thrsh = 2)
        doC_ngramed = {'{}'.format(p):make_ngram(c, ngram, mincnt = 1, thrsh = 1) for p, c in doC.items()}

    #     3. Fit LDA and tune parameters!

    #     3.1. Prepare doLDA instance and hyper parameters
        inst = doLDA(corp_forLDA, use_tfidf=False, top_n_words=20)
        inst.get_bow(no_below = .0, no_above = 1)

    #     3.2. Tune, get topic DF and optimal solutions
        flda, topicDF, optparset = LDA_tuning(inst_doLDA=inst,
                                              weight_on_coher=1.,
                                              ntopics= tpc_grd,
                                              hypars = {'alpha':alpha, 'beta':['auto']},
                                              rand = random_seed,
                                              iters = 2*10**2)
        plt.close()

        #     4. Save files
        if save:
            tops = fitdocs(inst, flda, doC_ngramed)
            results = {'project':project, 'flda':flda, 'inst':inst, 'optparset':optparset, 'ngram':ngram, 'random_seed':random_seed}
            tmstmp = save_results(mkdir = False, write = True, ldavis = False, **results)
            GSAPI.MakeSheet({'topicDF':topicDF, 'topics_top5':tops.fillna('')}, sheet_title = tmstmp + '_' + project, RANGE_NAME = 'A1:AA1000')

In [328]:
for i in tqdm(range(5)):
    Main(False)
print('============\nResults saved!\n============')

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Sheet "200427_2325_21대_총선" successfully Created


 20%|████████████████▌                                                                  | 1/5 [04:11<16:44, 251.10s/it]

Sheet "200427_2330_21대_총선" successfully Created


 40%|█████████████████████████████████▏                                                 | 2/5 [08:12<12:24, 248.05s/it]

...

Sheet "200427_2338_21대_총선" successfully Created


 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [16:26<04:07, 247.92s/it]

Sheet "200427_2342_21대_총선" successfully Created


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [20:35<00:00, 247.06s/it]


Results saved!
